# Hit Song Predictor: A Logistic Regression Approach

## Project Overview
This project uses logistic regression to predict whether a song will be a 'hit' based on audio features like danceability, energy, valence, and tempo. The model analyzes patterns in successful tracks to identify what makes a song popular.

**Key Skills Demonstrated:**
- Binary classification with logistic regression
- Feature engineering and selection
- Handling imbalanced datasets
- Model evaluation and interpretation
- Data visualization for music analytics

### Import Necessary Tools & Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# Set style for better visualizations
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

### Load the Dataset

In [ ]:
df = pd.read_csv("songs_normalize.csv")

### Inspect Data

In [ ]:
df.info()

In [ ]:
df.head()

### Exploratory Data Analysis

In [ ]:
# Check for missing values
print("Missing Values:")
print(df.isnull().sum())
print("\nData Types:")
print(df.dtypes)
print("\nBasic Statistics:")
df.describe()

In [ ]:
# Define what makes a 'hit' - songs with popularity > 70
# Adjust this threshold based on dataset
popularity_threshold = df['popularity'].quantile(0.75)
df['is_a_hit'] = (df['popularity'] > popularity_threshold).astype(int)

print(f"Popularity threshold for 'hit': {popularity_threshold}")
print(f"\nClass distribution:")
print(df['is_a_hit'].value_counts())
print(f"\nHit rate: {df['is_a_hit'].mean():.2%}")

### Visualize audio feature distributions for hits vs non-hits

#### The Visualization shows
Expected Strong Features:

- Danceability ⭐⭐⭐
- Energy ⭐⭐⭐
- Valence ⭐⭐

Expected Weak Features:

- Tempo, Acousticness, Instrumentalness, Liveness, Speechiness, and Loudness

In [ ]:
audio_features = ['danceability', 'energy', 'valence', 'tempo', 'acousticness', 
                  'instrumentalness', 'liveness', 'speechiness', 'loudness']

fig, axes = plt.subplots(3, 3, figsize=(15, 12))
axes = axes.ravel()

for idx, feature in enumerate(audio_features):
    axes[idx].hist(df[df['is_a_hit']==1][feature], alpha=0.5, label='Hit', bins=30, color='green')
    axes[idx].hist(df[df['is_a_hit']==0][feature], alpha=0.5, label='Not a Hit', bins=30, color='red')
    axes[idx].set_title(f'{feature.capitalize()} Distribution')
    axes[idx].legend()
    axes[idx].set_xlabel(feature)
    axes[idx].set_ylabel('Frequency')

plt.tight_layout()
plt.savefig('feature_distributions.png', dpi=300, bbox_inches='tight')
plt.show()

### Correlation heatmap

In [ ]:
plt.figure(figsize=(12, 10))
correlation_matrix = df[audio_features + ['is_a_hit']].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            fmt='.2f', square=True, linewidths=1)
plt.title('Feature Correlation Heatmap', fontsize=16, fontweight='bold')
plt.savefig('correlation_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()

## Correlation Analysis

The correlation heatmap reveals an that no individual audio feature shows string correlation with hit status (as expected since music is subjective, and there are many factors that go into having a hit, such as marketing, artist's popularity, timing, and cultureal context). The strongest correlations are:

- Danceability: -0.02
- Energy: -0.07  
- Valence: -0.09
- Acousticness: 0.07
- All other features: ≈ 0.00

### My Solution: Feature Engineering

Since individual features lack predictive power, I will create interaction features that combine multiple attributes:

1. **Energy × Danceability**: Captures the "party factor" - songs that are both energetic & danceable
2. **Valence × Energy**: Represents upbeat, positive vibes
3. **Mood Score**: Average of valence and energy for overall emotional tone
4. **Normalized Tempo**: Standardized BPM for better model scaling

**Hypothesis**: These engineered features will capture the non-linear relationships and feature combinations that differentiate hits from non-hits, where individual features alone cannot.